# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sitka,57.0531,-135.3300,285.09,87,75,3.60,US,1686343239
1,1,aasiaat,68.7098,-52.8699,273.98,83,100,6.06,GL,1686343204
2,2,adamstown,-25.0660,-130.1015,294.38,60,52,7.07,PN,1686343327
3,3,qaqortoq,60.7167,-46.0333,279.03,68,70,1.58,GL,1686343205
4,4,kandiari,26.9156,68.5216,304.36,59,0,9.49,PK,1686343328


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot

cities_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
cities_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"]<300.15)&(city_data_df["Max Temp"]>294.15)&(city_data_df["Cloudiness"]==0)&(city_data_df["Wind Speed"]<4.5)]
# Drop any rows with null values
ideal_cities = ideal_cities.dropna()
# Display sample data
ideal_cities.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,riasi,33.0833,74.8333,299.68,47,0,3.74,IN,1686343344
52,52,esmeraldas,-19.7625,-44.3139,298.63,49,0,1.59,BR,1686343350
102,102,toliara,-23.3500,43.6667,294.24,83,0,2.06,MG,1686343370
128,128,dzhalagash,45.0833,64.6667,295.42,27,0,4.17,KZ,1686343382
145,145,colonia,50.9333,6.9500,298.05,51,0,3.60,DE,1686343156


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
38,riasi,IN,33.0833,74.8333,47,
52,esmeraldas,BR,-19.7625,-44.3139,49,
102,toliara,MG,-23.3500,43.6667,83,
128,dzhalagash,KZ,45.0833,64.6667,27,
145,colonia,DE,50.9333,6.9500,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
riasi - nearest hotel: No hotel found
esmeraldas - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
dzhalagash - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
arraial do cabo - nearest hotel: No hotel found
pollock pines - nearest hotel: North Canyon Inn Bed & Breakfast
carloforte - nearest hotel: Hotel Riviera
roses - nearest hotel: Ramblamar
paphos - nearest hotel: Mayfair Hotel
orchard homes - nearest hotel: Broadway Inn
antonio enes - nearest hotel: Hotel Quirimbas
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera


,City,Country,Lat,Lng,Humidity,Hotel Name
38,riasi,IN,33.0833,74.8333,47,No hotel found
52,esmeraldas,BR,-19.7625,-44.3139,49,No hotel found
102,toliara,MG,-23.3500,43.6667,83,Ambary
128,dzhalagash,KZ,45.0833,64.6667,27,No hotel found
145,colonia,DE,50.9333,6.9500,51,Wasserturm Hotel Cologne
188,arraial do cabo,BR,-22.9661,-42.0278,73,No hotel found
234,pollock pines,US,38.7613,-120.5866,50,North Canyon Inn Bed & Breakfast
305,carloforte,IT,39.1450,8.3058,91,Hotel Riviera
313,roses,ES,42.2620,3.1769,87,Ramblamar
446,paphos,CY,34.7667,32.4167,78,Mayfair Hotel


In [9]:
hotel_df.to_csv("output_data/hotels.csv", index_label="H_City_ID")

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=['Country','Hotel Name'],
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)
# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)